In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.preprocessing as sp
import pickle
import matplotlib.pyplot as plt
import os
from datetime import date
today = date.today()
from utils import preprocessing,meanProfileAnalysis
from utils.impactscore import impact_score_wt_mt, impact_score_wt_mt_perplate
from utils.feature_importance import generate_feature_importance_figures
from utils.subpopulation_analysis import generate_subpopulation_analysis_figures


# Steps:

### 1- Clean metadata and make a columns names uniform to input to the pipeline
   - Done in standardize_platemaps notebook

### 2. Save a subset of intensity features 
   - for transfection efficiency exploration and parameter selection of transfection detection

### 3. Read Intensity features and save their distribution for transfected versus untransfected wells 
   - for transfection efficiency exploration and parameter selection of transfection detection
   
### 4. Fix transfection detection parameters and generate and save population level profiles
   - Based on the fixed parameters for transfection detection, generate and save population level profiles and also save transfected single cells for visualization and subpopulation analysis
   
### 5. Load and preprocess replicate level profiles 
   - read the saved per-well population profiles
     - cpg0026-lacoste_haghighi-rare-diseases/broad/workspace/population_profiles
   
### 6. Calculate replicate correlation of profiles
   - Calculate technical replicate reproducibility using correlation coefficient metric and compare with
     random-pair replicability (null distribution)

   
### 7. Calculate WT-MT impact scores and save the to results

   

### 8. (optional) generate plots for WT-MT impact subpopulation analysis


## Read cleaned metadata from "metadata/reprocessed" folder

In [ ]:
rootDir='/home/ubuntu/gallery/cpg0026-lacoste_haghighi-rare-diseases/broad/workspace'
rootSaveDir='./results/'

batch_12='set1_set2'
annot_df=pd.read_csv(rootDir+'/metadata/reprocessed/'+batch_12+'.csv')
annot_df.head()

## 2. Save a subset of intensity features 

In [ ]:

########################## 
channels_used=['Protein']

listOfBatchPlates=annot_df.Metadata_batch_Plate.unique().tolist();
for bp in listOfBatchPlates:
    _=preprocessing.saveRawIntensityFeatures(bp,annot_df,rootDir,channels_used);

## 3. Read Intensity features and save their distribution 
    - for transfected versus untransfected wells 

In [ ]:
######## READ
channels_used=['Protein']

df_inten,df_inten_scaled_perPlate=\
        utils.preprocessing.readNormalizeIntensityFeatures(annot_df,rootDir,channels_used);

In [ ]:
######## PLOT and SAVE
intFeatures=['Cells_Intensity_UpperQuartileIntensity_Protein',\
             'Cells_Intensity_MeanIntensity_Protein',\
             'Cells_Intensity_MaxIntensity_Protein']


plot_int_params={'log_scale_enabled':True,\
                 'sharex_enabled':False,\
                 'hist_n_bins':1000,\
                 'zero_offset_4logscale_plot':0.00001,\
                 'intFeature':intFeatures[1],
                 'hue_column':'Metadata_transfection'
                }

listOfBatches=annot_df.batch.unique().tolist();

for inti in intFeatures:
    plot_int_params['intFeature']=inti
    f=utils.preprocessing.plot_intensity_features(listOfBatches,[df_inten,df_inten_scaled_perPlate],plot_int_params)

    saveDir=rootSaveDir+'/results/intensity_dists/'+batch_12
    os.makedirs(saveDir, exist_ok=True)
    f.savefig(saveDir+'/'+inti+'.png')


## 4. Generate population profiles

In [ ]:
precomputed_params_batches= {'Maxproj_Replicates_Original_Screen':[0.0033,0.023],\
'Maxproj_Kinase_Plates':[0.0018,0.0234],\
'Maxproj_Common_Variants':[0.0097,0.0225],\
'Maxproj_Cancer_Mutations_Screen':[0.0065,0.0228],\
'PILOT_1_maxproj':[0.005,0.0165]}



transfection_params_dict={'Method':'single_intensity_feature_thrsh',\
                              'intensity_feature_to_use':'Cells_Intensity_MeanIntensity_Protein',\
                              'thresholding_method': 'precomputed_batch_specific_thrsh',\
                              'pre_detection_scaler':'MinMax',\
                              'precomputed_params': []} 

feature_scaling_params_dict={'feature_scaler': 'Robust'}
all_params={}
all_params['enrichement_profiles_params']={}
all_params['transfection_params_dict']=transfection_params_dict
all_params['feature_scaling_params_dict']=feature_scaling_params_dict
all_params['save_single_cells']=True


listOfBatchPlates=annot_df.Metadata_batch_Plate.unique().tolist();

for bp in listOfBatchPlates:
    b=bp.split('-')[0]
    all_params['transfection_params_dict']['precomputed_params']=precomputed_params_batches[b]
    preprocessing.generate_population_profiles(bp,annot_df,rootDir,all_params);

## 5. Load and preprocess replicate level profiles 


In [ ]:
# sc_per_plate_scaling # 'sc_scaled_per_plate','raw'
# zscored_profiles # 'untransfected','untransfected_stringent'
feature_scaling_params_dict={'sc_per_plate_scaling':'sc_scaled_per_plate',\
                             'zscored_profiles':[False,'untransfected'],\
                             'post_scale_all_profiles':[True,'Standard']} 

dirs_params_dict={'rootDir':rootDir,\
                  'profiles_folder_in_workspace': 'population_profiles'}
read_pop_params={}
read_pop_params['dirs_params_dict']=dirs_params_dict
read_pop_params['feature_scaling_params_dict']=feature_scaling_params_dict
read_pop_params['protein_channel_suffix']='Protein'

df_scaled_annot,cpFeats_A,cpFeats_P,cpFeats_NP=\
meanProfileAnalysis.read_merge_preprocess_meanProfiles(annot_df,read_pop_params);

# dfTransSummary = df_scaled_annot[['Metadata_batch_Plate','Metadata_Sample_Unique','n_transf','n_untransf','transf_Ratio']]
dfTransSummary=df_scaled_annot[annot_df.columns.tolist()+['n_transf','n_untransf','transf_Ratio']];

In [ ]:
len(cpFeats_A),len(cpFeats_P),len(cpFeats_NP)

#### Filterpot wells that:
- Manual annotation of "not transfected"
- Detected number of transfected cells ("n_transf" column) less than x

In [ ]:
df_scaled_annot=df_scaled_annot[df_scaled_annot['Metadata_transfection']==False]
df_scaled_annot=df_scaled_annot[df_scaled_annot['n_transf']>5].reset_index(drop=True)


In [ ]:

df_rep_level_scaled.groupby(['Metadata_Sample_Unique']).size().reset_index()

## 6. Calculate replicate correlation of profiles
   - Save curve plots and values to results/replicate_corr_curves

In [ ]:
from singlecell.process.replicate_correlation import replicate_null_corr_coefs
from singlecell.process import normalize_funcs

df_rep_level=df_scaled_annot[df_scaled_annot['transfection_status']==1].reset_index(drop=True)

df_rep_level_scaled=normalize_funcs.standardize_per_catX(df_rep_level,'Metadata_batch_Plate',cpFeats_P+cpFeats_NP).copy();
# df_rep_level_scaled = normalize_funcs.standardize_df_columns(df_rep_level,cpFeats_P+cpFeats_NP,'Standard')

nOfReps=df_rep_level_scaled.groupby(['Metadata_Sample_Unique']).size().reset_index()
pairWithReplicates=nOfReps.loc[nOfReps[0]!=1,:].reset_index()['Metadata_Sample_Unique']#.groupby([0]).size()

scal_status=df_rep_level_scaled['normalization'].unique()[0]
zscor_status=df_rep_level_scaled['zscored'].unique().astype(str)[0]
# if not np.isnan(df_rep_level_scaled['zscored'].unique()[0])

saveDir=rootSaveDir+'/results/replicate_corr_curves/'+batch_12
# 
os.makedirs(saveDir, exist_ok=True)

pertColName='Metadata_Sample_Unique'
repCor4impactList=[]
rep_corr_ls=[];null_corr_ls=[]
for f,ch,t in zip([cpFeats_P,cpFeats_NP],['p','np'],['Protein_Channel','NonProtein_Channels']):
    print(ch,t)
    t2=t+', '+scal_status+', zscored: '+zscor_status
    fh_2save,repCorrDf,a,b=replicate_null_corr_coefs(df_rep_level_scaled,pertColName,f,1,title=t2,hist_bins=100)
    fh_2save.savefig(saveDir+'/'+ch+'_'+scal_status+'_'+zscor_status+'_'+today.strftime("%Y%m%d")+'.png')
    repCorrDf=repCorrDf.add_suffix('_'+ch)
    rep_corr_ls.append(a);
    null_corr_ls.append(b);
    repCor4impactList.append(repCorrDf);
    
repCorr_df_avg=pd.concat(repCor4impactList,axis=1).reset_index().rename(columns={'index':pertColName})
if 0:
    repCorr_df_avg.to_csv(saveDir+'/'+scal_status+'_'+zscor_status+'_'+today.strftime("%Y%m%d")+'.csv',index=False)
df_rep_level_scaled=pd.merge(df_rep_level_scaled,repCorr_df_avg,how='left',on=pertColName)

In [ ]:
rep_reprod_cc_curves_source_numbers=pd.DataFrame()
rep_reprod_cc_curves_source_numbers['rep_corr_p']=rep_corr_ls[0]
rep_reprod_cc_curves_source_numbers['null_corr_p']=null_corr_ls[0]

rep_reprod_cc_curves_source_numbers['rep_corr_np']=rep_corr_ls[1]
rep_reprod_cc_curves_source_numbers['null_corr_np']=null_corr_ls[1]

In [ ]:
rep_reprod_cc_curves_source_numbers.to_csv('rep_curves_source_data.csv')

### 7. Calculate WT-MT impact scores and save
- Approach 1: average replicate level profiles and score treatment level profiles
- Approach 2: calculate impact scores per plate

In [ ]:
# Approach 1
wt_mt_cols=['Gene','Metadata_Sample_Unique']
impact_scores_trt_profs = impact_score_wt_mt(df_rep_level_scaled,repCorr_df_avg,[cpFeats_P,cpFeats_NP],wt_mt_cols);

saveDir=rootSaveDir+'/results/Impact-Scores/Method-MeanProfiles/'+batch_12
os.makedirs(saveDir, exist_ok=True)
impact_scores_trt_profs.to_csv(saveDir+'/impact_scores_trt_'+today.strftime("%Y%m%d")+'.csv',index=False)

In [ ]:
# Approach 2
wt_mt_cols=['Gene','Metadata_Sample_Unique']
impact_scores_perplate = impact_score_wt_mt_perplate(df_rep_level_scaled,repCorr_df_avg,\
                                                     [cpFeats_P,cpFeats_NP],wt_mt_cols)

saveDir=rootSaveDir+'/results/Impact-Scores/Method-MeanProfiles/'+batch_12
os.makedirs(saveDir, exist_ok=True)
impact_scores_perplate.to_csv(saveDir+'/impact_scores_perplate_'+today.strftime("%Y%m%d")+'.csv',index=False)

### 8. (optional) generate plots for WT-MT impact subpopulation analysis

In [ ]:
###### Feature importance
save_dir = rootSaveDir + '/results/WT-MT-same-plates-p'
# Channelss=['Protein','Mito','ER','DNA']
channels=['Protein']
incompPairs=generate_feature_importance_figures(df_rep_level_scaled, cpFeats_P, save_dir, channels);

In [ ]:
###### generate subpopulation analysis
save_dir = rootSaveDir + '/results/WT-MT-same-plates-p'

nSampleSCs=6;
boxSize=200;
n_clusters=10

params={'results_dir':'',\
        'control_label':'Wild-type',\
        'pert_label': 'Mutant',\
        'n_clusters':n_clusters,\
        'n_cells_4single_cell_plots':nSampleSCs,\
        'channels_4single_cell_plots':['Protein','DNA','Mito','ER'],\
        'boxSize_4single_cell_plots': boxSize,\
        'compressed_ims_flag':False,\
        'img_save_format':'.png'}     
                
plt.ioff()
incompPairs=generate_subpopulation_analysis_figures(df_rep_level_scaled, cpFeats_P,rootDir, save_dir, params);
